**Общая информация**

**Срок сдачи:** 13 марта 2017, 06:00 <br\>
**Штраф за опоздание:** -2 балла после 06:00 13 марта, -4 балла после 06:00 20 марта, -6 баллов после 06:00 27 марта

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack создать таск в приватный чат:
/todo Фамилия Имя ссылка на гитхаб @alkhamush
Пример:
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе.

###### Задание 2 (2 балла)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn 4 балла для iris и mnist
Для этого используем numpy

###### Задание 3 (2 балла)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 3 (2 балла)
Выполнить требования pep8

###### Задание 5 (2 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [5]:
%%pycodestyle


class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):

    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def get_n_nearest_labbels(self, X):
        dist = np.array([np.sqrt(((self.X_train - vector) ** 2).sum(axis=1))
                         for vector in np.float64(X)])
        sorted_indexes = dist.argsort(axis=1)

        n_nearest_indexes = sorted_indexes[:, : self.n_neighbors]
        n_nearest_labbels = self.y_train[n_nearest_indexes]
        return n_nearest_labbels

    def most_frequent(self, row):
        return max(Counter(row).items(), key=lambda item: item[1])[0]

    def predict(self, X):
        n_nearest_labbels = self.get_n_nearest_labbels(X)
        predictions = np.array([self.most_frequent(row)
                                for row in n_nearest_labbels])
        return predictions

    def predict_proba(self, X):
        n_nearest_labbels = self.get_n_nearest_labbels(X)
        classes = list(set(self.y_train))
        classes.sort()
        probabilitis = []
        for row in n_nearest_labbels:
            counter = Counter(row)
            amount = sum(counter.values())
            row_probability = []
            for labbel in classes:
                if labbel in classes:
                    row_probability.append(counter[labbel] / amount)
                else:
                    row_probability.append(0)
            probabilitis.append(row_probability)
        return np.array(probabilitis)

    def score(self, X, y):
        predictions = self.predict(X)
        score = predictions == y
        return score.sum() / y.size

**IRIS**

In [318]:
iris = datasets.load_iris()

In [319]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [320]:
clf = KNeighborsClassifier(n_neighbors=3)
my_clf = MyKNeighborsClassifier(n_neighbors=3)

In [321]:
%time clf.fit(X_train, y_train)

CPU times: user 2.89 ms, sys: 0 ns, total: 2.89 ms
Wall time: 2.09 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [322]:
%time my_clf.fit(X_train, y_train)

CPU times: user 16 µs, sys: 3 µs, total: 19 µs
Wall time: 25.7 µs


In [323]:
%time clf.predict(X_test)

CPU times: user 1.98 ms, sys: 374 µs, total: 2.35 ms
Wall time: 2.08 ms


array([0, 1, 1, 2, 2, 0, 2, 0, 1, 0, 2, 2, 2, 0, 2])

In [324]:
print(y_test)
%time my_clf.predict(X_test)

[0 1 1 2 2 0 2 0 1 0 2 2 1 0 1]
CPU times: user 5.34 ms, sys: 254 µs, total: 5.59 ms
Wall time: 4.35 ms


array([0, 1, 1, 2, 2, 0, 2, 0, 1, 0, 2, 2, 2, 0, 2])

In [325]:
%time clf.predict_proba(X_test)

CPU times: user 1.92 ms, sys: 294 µs, total: 2.22 ms
Wall time: 1.88 ms


array([[1.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.33333333, 0.66666667],
       [1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.        , 1.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ]])

In [326]:
%time my_clf.predict_proba(X_test)

CPU times: user 3.56 ms, sys: 673 µs, total: 4.23 ms
Wall time: 3.13 ms


array([[1.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.33333333, 0.66666667],
       [1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 0.        , 1.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ]])

In [327]:
clf.score(X_test, y_test)

0.8666666666666667

In [328]:
my_clf.score(X_test, y_test)

0.8666666666666667

In [218]:
# Задание 3

**MNIST**

In [88]:
mnist = fetch_mldata('MNIST original')

In [271]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.001, stratify=mnist.target)

In [272]:
clf = KNeighborsClassifier(n_neighbors=5)
my_clf = MyKNeighborsClassifier(n_neighbors=5)
my_clf2 = MyKNeighborsClassifier2(n_neighbors=5)

In [273]:
%time clf.fit(X_train, y_train)

CPU times: user 2min, sys: 1.41 s, total: 2min 2s
Wall time: 2min 38s


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [274]:
%time my_clf.fit(X_train, y_train)

CPU times: user 16 µs, sys: 2 µs, total: 18 µs
Wall time: 24.1 µs


In [275]:
print(y_test)
%time clf.predict(X_test)

[8. 1. 0. 7. 3. 4. 9. 4. 6. 8. 2. 6. 1. 9. 9. 6. 4. 6. 5. 3. 4. 4. 5. 1.
 3. 3. 0. 5. 2. 6. 8. 2. 1. 7. 6. 0. 4. 8. 0. 2. 9. 7. 7. 3. 1. 1. 7. 5.
 8. 5. 9. 2. 9. 8. 2. 5. 0. 1. 7. 9. 8. 6. 2. 1. 3. 7. 0. 3. 0. 4.]
CPU times: user 20.3 s, sys: 14.9 ms, total: 20.3 s
Wall time: 20.6 s


array([8., 1., 0., 7., 3., 4., 9., 4., 6., 8., 2., 6., 1., 9., 9., 6., 4.,
       6., 5., 3., 4., 4., 5., 1., 3., 3., 0., 5., 2., 6., 8., 2., 1., 7.,
       6., 0., 4., 8., 0., 2., 9., 7., 7., 3., 1., 1., 7., 5., 7., 5., 9.,
       2., 9., 8., 2., 5., 0., 1., 9., 9., 8., 6., 2., 1., 3., 7., 0., 3.,
       0., 4.])

In [276]:
%time my_clf.predict(X_test)

CPU times: user 30.3 s, sys: 24 s, total: 54.2 s
Wall time: 1min 25s


array([8., 1., 0., 7., 3., 4., 9., 4., 6., 8., 2., 6., 1., 9., 9., 6., 4.,
       6., 5., 3., 4., 4., 5., 1., 3., 3., 0., 5., 2., 6., 8., 2., 1., 7.,
       6., 0., 4., 8., 0., 2., 9., 7., 7., 3., 1., 1., 7., 5., 7., 5., 9.,
       2., 9., 8., 2., 5., 0., 1., 9., 9., 8., 6., 2., 1., 3., 7., 0., 3.,
       0., 4.])

In [278]:
%time clf.predict_proba(X_test)

CPU times: user 19.7 s, sys: 144 ms, total: 19.8 s
Wall time: 20 s


array([[0.2, 0. , 0. , 0. , 0. , 0.2, 0. , 0. , 0.6, 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 0. , 0.8, 0. , 0. , 0. , 0.2, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. 

In [279]:
%time my_clf.predict_proba(X_test)

CPU times: user 29.5 s, sys: 23.1 s, total: 52.6 s
Wall time: 53 s


array([[0.2, 0. , 0. , 0. , 0. , 0.2, 0. , 0. , 0.6, 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 0. , 0.8, 0. , 0. , 0. , 0.2, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. 

In [280]:
clf.score(X_test, y_test)

0.9714285714285714

In [281]:
my_clf.score(X_test, y_test)

0.9714285714285714

Задание 5

In [ ]:
# seaborn are based on matplotlib for visualization static data
# matplotlib is for visualization 2D and 3D graphics
# train_test_split partitions dataset on train dataset and test datset
# Pipeline%load_ext (from sklearn.pipeline import Pipeline) loads 
# Pipeline which is sequentially apply a list of transforms 
# and a final estimator.
# StandardScaler (from sklearn.preprocessing import StandardScaler) 
# standardizes features by removing the mean and scaling to 
# unit variance
# ClassifierMixin is a class for all classifiers in scikit-learn 
# and contains a method score that returns the mean accuracy 
# on the given test data and labels.
# NeighborsBase is base class for nearest neighbors estimators
# KNeighborsMixin is Mixin for k-neighbors searches
# SupervisedIntegerMixin is a class for all classifiers in scikit-learn
# and contains method fit that fits the model using X as training data 
# and y as target values